In [1]:
from datetime import datetime

import scenarios
from dotenv import load_dotenv
from jinja2 import Template
from util import short_uid
from kubernetes import client, config
import yaml

load_dotenv()

config.load_kube_config(
    config_file="/home/silv/exp-cluster/config.yaml")

apps_v1 = client.AppsV1Api()
core_v1 = client.CoreV1Api()


In [2]:
with open('dummy-model.yaml.jinja2', "r") as f:
    template = Template(f.read())


def deploy_model(model_id: int, node_id: str, id_prefix: str):
    model_deployment_id = f"{id_prefix}-{model_id}-{short_uid()}"
    data = scenarios.DUMMY_MODELS[model_id]
    model_template = template.render(
        data | {"model_id": model_deployment_id, "node_id": node_id}
    )
    docs = yaml.safe_load_all(model_template)
    for doc in docs:
        if doc.get("kind") == "Deployment":
            response = apps_v1.create_namespaced_deployment(namespace="mulambda",
                                                            body=doc)
            print("Deployment created. Status='%s'" % str(response.status))
        elif doc.get("kind") == "Service":
            response = core_v1.create_namespaced_service(namespace="mulambda",
                                                         body=doc)
            print("Service created. Status='%s'" % str(response.status))


In [4]:
SCENARIO = 'parity'

In [5]:
deployments = scenarios.SCENARIOS[SCENARIO]
id_prefix = f"{SCENARIO}-{datetime.now().strftime('%Y%m%d%H%M%S')}"
for node_id, models in enumerate(deployments):
    for model_id in models:
        print(f"Deploying model {model_id} on node {node_id}")
        deploy_model(model_id, node_id, id_prefix)
        print("Done")

Deploying model 0 on node 0
Deployment created. Status='{'available_replicas': None,
 'collision_count': None,
 'conditions': None,
 'observed_generation': None,
 'ready_replicas': None,
 'replicas': None,
 'unavailable_replicas': None,
 'updated_replicas': None}'
Service created. Status='{'load_balancer': {'ingress': None}}'
Done
Deploying model 4 on node 0
Deployment created. Status='{'available_replicas': None,
 'collision_count': None,
 'conditions': None,
 'observed_generation': None,
 'ready_replicas': None,
 'replicas': None,
 'unavailable_replicas': None,
 'updated_replicas': None}'
Service created. Status='{'load_balancer': {'ingress': None}}'
Done
Deploying model 5 on node 0
Deployment created. Status='{'available_replicas': None,
 'collision_count': None,
 'conditions': None,
 'observed_generation': None,
 'ready_replicas': None,
 'replicas': None,
 'unavailable_replicas': None,
 'updated_replicas': None}'
Service created. Status='{'load_balancer': {'ingress': None}}'
Done
D